# Get the Exact Answer
Start off by computing the exact Hessian to use a reference point. 
First relax the structure then compute the Hessians using [ase's Vibrations module](https://databases.fysik.dtu.dk/ase/ase/vibrations/modes.html), which will compute them numerically using central derivatives

In [1]:
from ase.thermochemistry import IdealGasThermo
from ase.vibrations import VibrationsData, Vibrations
from ase.calculators.psi4 import Psi4
from ase.optimize import QuasiNewton
from ase import Atoms, units
from ase.io import write
from time import perf_counter
from platform import node
from pathlib import Path
import numpy as np
import shutil
import json
import os

Configuration

In [32]:
molecule_name = 'ethanol'
method = 'wb97x-d'
basis = 'cc-pvtz'
threads = min(os.cpu_count(), 12)

Derived

In [33]:
run_name = f'{molecule_name}_{method}_{basis}'

## Load in Target Molecule
We have it in a JSON file from PubChem

In [34]:
def load_molecule(name: str) -> Atoms:
    """Load a molecule from a PubChem JSON file
    
    Args:
        name: Name of the molecule
    Returns:
        ASE Atoms object
    """
    
    # Get the compound data
    with open(f'data/structures/{name}.json') as fp:
        data = json.load(fp)
    data = data['PC_Compounds'][0]
        
    # Extract data from the JSON
    atomic_numbers = data['atoms']['element']
    positions = np.zeros((len(atomic_numbers), 3))
    conf_data = data['coords'][0]['conformers'][0]
    for i, c in enumerate('xyz'):
        if c in conf_data:
            positions[:, i] = conf_data[c]
        
    # Build the object    
    return Atoms(numbers=atomic_numbers, positions=positions)

In [35]:
atoms = load_molecule(molecule_name)

## Perform the Geometry Optimization
Build the ASE calculator then run QuasiNewton to a high tolerance

In [36]:
calc = Psi4(method=method, basis=basis, num_threads=threads, memory='4096MB')

In [37]:
%%time
atoms.calc = calc
dyn = QuasiNewton(atoms)
dyn.run(fmax=0.01)

                Step[ FC]     Time          Energy          fmax
BFGSLineSearch:    0[  0] 14:30:14    -4219.144408        0.7930
BFGSLineSearch:    1[  2] 14:30:31    -4219.158073        0.5048
BFGSLineSearch:    2[  4] 14:30:49    -4219.168016        0.3333
BFGSLineSearch:    3[  6] 14:31:07    -4219.173063        0.1898
BFGSLineSearch:    4[  8] 14:31:24    -4219.176127        0.2506
BFGSLineSearch:    5[ 10] 14:31:42    -4219.180164        0.1618
BFGSLineSearch:    6[ 12] 14:31:59    -4219.181108        0.0880
BFGSLineSearch:    7[ 14] 14:32:16    -4219.181824        0.1177
BFGSLineSearch:    8[ 16] 14:32:33    -4219.182404        0.0521
BFGSLineSearch:    9[ 17] 14:32:42    -4219.182678        0.0919
BFGSLineSearch:   10[ 19] 14:32:59    -4219.182811        0.0259
BFGSLineSearch:   11[ 20] 14:33:08    -4219.182914        0.0207
BFGSLineSearch:   12[ 22] 14:33:26    -4219.182940        0.0211
BFGSLineSearch:   13[ 24] 14:33:43    -4219.182970        0.0185
BFGSLineSearch:   14[ 25]

True

Save the output file

In [38]:
out_dir = Path('data') / 'exact'
out_dir.mkdir(exist_ok=True)

In [39]:
write(out_dir / f'{run_name}.xyz', atoms)

## Compute the Hessian using ASE
ASE has a built-in method which uses finite displacements

In [40]:
if Path('vib').is_dir():
    shutil.rmtree('vib')

In [41]:
%%time
finite_diff_time = perf_counter()
vib = Vibrations(atoms)
vib.run()
finite_diff_time = perf_counter() - finite_diff_time

CPU times: user 53min 15s, sys: 1min 35s, total: 54min 51s
Wall time: 5min 6s


Save the vibration data

In [42]:
vib_data = vib.get_vibrations()
with (out_dir / f'{run_name}-ase.json').open('w') as fp:
    vib_data.write(fp)

Print the ZPE for reference

In [43]:
vib_data.get_zero_point_energy()

2.1884190925383207

## Repeat with Psi4's analytic derivatives
See if we get the same answer faster

In [44]:
analytic_time = perf_counter()
calc.set_psi4(atoms)
hess = calc.psi4.hessian(f'{method}/{basis}')
analytic_time = perf_counter() - analytic_time

In [45]:
analytic_hess = hess.to_array() * units.Hartree / units.Bohr / units.Bohr

Convert it to an ASE object and save

In [46]:
vib_data = VibrationsData.from_2d(atoms, analytic_hess)

In [47]:
vib_data = vib.get_vibrations()
with (out_dir / f'{run_name}-psi4.json').open('w') as fp:
    vib_data.write(fp)

Print the ZPE for reference

In [48]:
vib_data.get_zero_point_energy()

2.1884190925383207

Save the runtimes

In [49]:
with (out_dir / f'{run_name}-times.json').open('w') as fp:
    json.dump({
        'hostname': node(),
        'finite-diff': finite_diff_time,
        'analytic': analytic_time,
    }, fp)

In [50]:
print(finite_diff_time)
print(analytic_time)

306.2348163090646
300.2469820249826
